In [2]:
import cv2
from pathlib import Path
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import imutils

%reload_ext autoreload
%autoreload 2
from main import (getROIFromVideo, cropWithROI, getOutputVidFrameSize, CalibWindow, 
                  getFrameFromVid, SelectionWindow, showImg)

# Crop

In [ ]:
vidPath = Path("data/screw/screw.mp4")
crop_roi = getROIFromVideo(str(vidPath))
cv2.destroyAllWindows()

In [ ]:
# import pickle
# f = open("temp_crop.pkl", "wb")
# pickle.dump(crop_roi, f)

# f = open("temp.pkl", "rb")
# pickle.load(f)

# Perspective

In [ ]:
VID_PATH = Path("data/screw/C0003.mp4")
vs = cv2.VideoCapture(str(VID_PATH))
ret, frame = vs.read()

perspectiveWindow = SelectionWindow("Perspective", frame)
perspectiveWindow.displayWindow()

In [ ]:
objLength = 1.83
objWidth = 0.6
imgWidth = 200
imgHeight = round(objLength/objWidth*imgWidth)

srcPts = np.float32(perspectiveWindow.selectionPts)
dstPts = np.float32([(0, 0), (imgWidth, 0), (imgWidth, imgHeight), (0, imgHeight)])
M = cv2.getPerspectiveTransform(srcPts, dstPts)
dst = cv2.warpPerspective(frame, M, (imgWidth,imgHeight))

plt.imshow(cv2.cvtColor(dst, cv2.COLOR_BGR2RGB))

# Track

In [26]:
VID_PATH = Path("data/pancake/IMG_2902.mp4")
OUTPUT_VID_PATH = VID_PATH.parent/(VID_PATH.stem+"_output.mp4")
OUTPUT_HEIGHT = 800
pipeline = lambda frame: frame
# pipeline = lambda frame: cv2.warpPerspective(frame, M, (imgWidth, imgHeight)) 
# pipeline = lambda frame: cropWithROI(frame, crop_roi)

IMSHOW_FLAG = True
WRITE_FLAG = True

cap = cv2.VideoCapture(str(VID_PATH))
fps = cap.get(cv2.CAP_PROP_FPS)

frameWidth, frameHeight = getOutputVidFrameSize(str(VID_PATH), pipeline, OUTPUT_HEIGHT)
out = cv2.VideoWriter(str(OUTPUT_VID_PATH), cv2.VideoWriter_fourcc(*"mp4v"), cap.get(cv2.CAP_PROP_FPS), (frameWidth,frameHeight))

totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
frameCount = 0
startTime = time.time()
saveFrames = []
saveCnts = []
centroids = []

ret, frame = cap.read()
while ret:
    if frameCount % 100 == 0 and frameCount != 0:
        elapsedTime = time.time()-startTime
        estTimeLeft = (totalFrames-frameCount)/frameCount*elapsedTime
        print(f"Frame {frameCount} out of {round(totalFrames)}.")
        print(f"\tTime taken: {round(elapsedTime)}s. Est. time left: {round(estTimeLeft)}s")

    frame = pipeline(frame)
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    (frame_h,frame_s,frame_v) = cv2.split(frame_hsv)

    # ret, th = cv2.threshold(frame_h, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    th = cv2.inRange(frame_h, 90, 120) & cv2.inRange(frame_s, 120, 255)
    cnts, hierarchy = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 

    if cnts:
        cnt = max(cnts, key=cv2.contourArea)
        M = cv2.moments(cnt)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        centroids.append((cX, cY))
        maxPts = 200
        dAlpha = 1/100
        firstPt = max(len(centroids)-maxPts, 0)
        alpha = max(maxPts-len(centroids), 0)*dAlpha
        for centroid in centroids[firstPt:]:
            frameCopy = frame.copy()
            cv2.circle(frameCopy, centroid, 5, (255,0,0), -1)
            frame = cv2.addWeighted(frame, 1-alpha, frameCopy, alpha, 0)
            alpha += dAlpha
        # cv2.drawContours(frame, [cnt], -1, (255, 0, 0), 1)
        saveFrames.append(frameCount)
        saveCnts.append(cnt)

    pts = np.array(centroids).reshape((-1, 1, 2))
    frame = cv2.polylines(frame, [pts],
                      False, (255,0,0), 5)

    
    frame = imutils.resize(frame, height=OUTPUT_HEIGHT)

    if WRITE_FLAG:
        out.write(frame)

    if IMSHOW_FLAG:
        cv2.imshow("Hue", frame_h)
        cv2.imshow("Saturation", frame_s)
        cv2.imshow("Value", frame_v)
        cv2.imshow("Detections", frame)
        key = cv2.waitKey(1)

        if key == ord('q') or key == ord('Q'):
            break

        if key == ord('p') or key == ord('P'):
            key = cv2.waitKey(0)
            if key == ord('p') or key == ord('P'): 
                continue
            
    ret, frame = cap.read()
    frameCount += 1

cv2.destroyAllWindows()
out.release()
cap.release()

Frame 100 out of 192.
	Time taken: 20s. Est. time left: 18s


In [ ]:
# import pickle
# f = open("temp_track.pkl", "wb")
# pickle.dump([saveFrames, saveCnts], f)

# f = open("temp.pkl", "rb")
# pickle.load(f)

# Get Calibration Scale

## Select Points

In [ ]:
VID_PATH = Path("data/screw/C0003.MP4")
pipeline = lambda frame: cropWithROI(frame, crop_roi)

frame = getFrameFromVid(str(VID_PATH), 1)
img_rgb = pipeline(frame)
calibLength = 1

CalibWin = CalibWindow("Calibration", img_rgb)
CalibWin.displayWindow()
calibPoints = CalibWin.calibPoints
scale = CalibWin.getCalibScale(calibLength)
key = cv2.waitKey(0)
print(f"Scale is {scale} pixels per metre")

## Image Length

In [ ]:
VID_PATH = Path("data/screw/C0003.MP4")
pipeline = lambda frame: cv2.warpPerspective(frame, M, (imgWidth, imgHeight)) 
frame = getFrameFromVid(str(VID_PATH), 1)
img_rgb = pipeline(frame)

objLength = 1.83
objWidth = 0.6

scale = img_rgb.shape[0]/objLength
print(f"Scale is {scale} pixels per metre")

# Write to CSV

In [ ]:
frames, xs, ys, ellipses = [], [], [], []

VID_PATH = Path("data/screw/C0003.MP4")
pipeline = lambda frame: cv2.warpPerspective(frame, M, (imgWidth, imgHeight)) 
frame = getFrameFromVid(str(VID_PATH), 1)
img_rgb = pipeline(frame)
rows,cols = img_rgb.shape[:2]

for frame, c in zip(saveFrames, saveCnts):
    moments = cv2.moments(c)
    if moments["m00"] > 0 and len(c) >= 5:
        xs.append(moments["m10"] / moments["m00"])
        ys.append(moments["m01"] / moments["m00"])
        frames.append(frame)
        
        ellipse = cv2.fitEllipse(c)
        # gradient = vy[0]/vx[0]
        ellipses.append(ellipse)

frames = np.array(frames)     
xs = np.array(xs)/scale
ys = np.array(ys)/scale

mask = (frames > 875) & (frames < 1375)

plt.plot(frames[mask], xs[mask])
plt.plot(frames[mask], ys[mask])
plt.show()
angles = np.array([e[2]-180 if e[2] > 90 else e[2] for e in ellipses])
plt.plot(frames[mask], angles[mask])

In [ ]:
csvPath = Path(VID_PATH).parent / (Path(VID_PATH).stem + ".csv")
df = pd.DataFrame([saveFrames, xs, ys, angles]).T
df.to_csv(csvPath, index=None, header=["frame","x","y","angle"])

# Get Calibration Scale

In [ ]:
# if getCalib: # Assume camera does not move between videos
#     # Set calibration length
#     CalibWin = CircleCalibWindow("Calibration", frame_trunc)
#     CalibWin.displayWindow()
#     scale = CalibWin.getCalibScale(calibRadius)


# Post-Process

In [37]:
cwd = Path("data/pancake/")
FILE_PATH = cwd/"IMG_2904.csv"
df = pd.read_csv(FILE_PATH, delimiter="\t", header=None)
df.columns = ["t", "x", "y"]
x = df["x"]
y = df["y"]

In [39]:
VID_PATH = cwd/"IMG_2904.mp4"
OUTPUT_VID_PATH = VID_PATH.parent/(VID_PATH.stem+"_output.mp4")
OUTPUT_HEIGHT = 800
pipeline = lambda frame: frame
# pipeline = lambda frame: cv2.warpPerspective(frame, M, (imgWidth, imgHeight)) 
# pipeline = lambda frame: cropWithROI(frame, crop_roi)

IMSHOW_FLAG = True
WRITE_FLAG = True

maxPts = 200
dAlpha = 1/100
alpha = 0.5

cap = cv2.VideoCapture(str(VID_PATH))
fps = cap.get(cv2.CAP_PROP_FPS)

frameWidth, frameHeight = getOutputVidFrameSize(str(VID_PATH), pipeline, OUTPUT_HEIGHT)
out = cv2.VideoWriter(str(OUTPUT_VID_PATH), cv2.VideoWriter_fourcc(*"mp4v"), cap.get(cv2.CAP_PROP_FPS), (frameWidth,frameHeight))

totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
frameCount = 0
startTime = time.time()
centroids = []

startFrameCount = 1

ret, frame = cap.read()
cols, rows, _ = frame.shape
while ret:
    if frameCount % 100 == 0 and frameCount != 0:
        elapsedTime = time.time()-startTime
        estTimeLeft = (totalFrames-frameCount)/frameCount*elapsedTime
        print(f"Frame {frameCount} out of {round(totalFrames)}.")
        print(f"\tTime taken: {round(elapsedTime)}s. Est. time left: {round(estTimeLeft)}s")

    if frameCount < startFrameCount:
        ret, frame = cap.read()
        frameCount += 1

    frame = pipeline(frame)

    if frameCount < len(x):
        centroids.append((int(x[frameCount]+rows//2), int(cols//2-y[frameCount])))

    frameCopy = frame.copy()

    pts = np.array(centroids).reshape((-1, 1, 2))
    frame = cv2.polylines(frame, [pts],
                      False, (255,0,0), 5)

    frame = cv2.addWeighted(frame, 1-alpha, frameCopy, alpha, 0)

    # firstPt = max(len(centroids)-maxPts, 0)
    # alpha = max(maxPts-len(centroids), 0)*dAlpha
    # for centroid in centroids[firstPt:]:
    #     frameCopy = frame.copy()
    #     cv2.circle(frameCopy, centroid, 5, (255,0,0), -1)
    #     frame = cv2.addWeighted(frame, 1-alpha, frameCopy, alpha, 0)
    #     alpha += dAlpha

    # cv2.drawContours(frame, [cnt], -1, (255, 0, 0), 1)
    
    frame = imutils.resize(frame, height=OUTPUT_HEIGHT)

    if WRITE_FLAG:
        out.write(frame)

    if IMSHOW_FLAG:
        cv2.imshow("Detections", frame)
        key = cv2.waitKey(1)

        if key == ord('q') or key == ord('Q'):
            break

        if key == ord('p') or key == ord('P'):
            key = cv2.waitKey(0)
            if key == ord('p') or key == ord('P'): 
                continue
            
    ret, frame = cap.read()
    frameCount += 1

cv2.destroyAllWindows()
out.release()
cap.release()

Frame 100 out of 326.
	Time taken: 2s. Est. time left: 4s
Frame 200 out of 326.
	Time taken: 4s. Est. time left: 2s
Frame 300 out of 326.
	Time taken: 5s. Est. time left: 0s


In [36]:
frameCount

203

# Image

In [39]:
# Import image
imgPath = Path("Image5.tif")
img = cv2.imread(str(imgPath))

# Crop
r = cv2.selectROI(img, fromCenter=False)
cv2.destroyAllWindows()


In [62]:
cropped = cropWithROI(img, r)

# Convert to grayscale
gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
# gray = cv2.GaussianBlur(gray, (5,5), 0)

# Iteratively increase contrast
iterations = 12
clahe = cv2.createCLAHE(clipLimit=1.1, tileGridSize=(12,12))
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))
tophat1 = gray.copy()
for i in range(iterations):
    cl1 = clahe.apply(tophat1)
    tophat1 = cv2.morphologyEx(cl1, cv2.MORPH_TOPHAT, cl1)


cv2.imwrite("gel_electrophoresis_enhanced.jpg", tophat1)
showImg(tophat1)

# # Morph to strips
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 1))
tophat1 = cv2.morphologyEx(tophat1, cv2.MORPH_OPEN, kernel)
cv2.imwrite("gel_electrophoresis_enhanced2.jpg", tophat1)

showImg(tophat1)

# Threshold
# tophat1 = cv2.GaussianBlur(tophat1, (5,5), 0)
ret, th = cv2.threshold(tophat1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Display
showImg(th)

In [62]:
cnts, _ = cv2.findContours(tophat1, cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

cv2.drawContours(cropped, cnts, -1, (0,0,255), 1)
showImg(cropped)

# th3 = cv2.adaptiveThreshold(tophat1,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,4)
# cv2.imshow("display", th3)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [107]:
th3 = cv2.adaptiveThreshold(cl1,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,4)
cv2.imshow("display", th3)
cv2.waitKey(0)
cv2.destroyAllWindows()